In [1]:
dia_inici_classes = '2025-09-15'
dia_final_classes = '2026-04-24'  # divendres anterior a inici de projecte DAW

festius = [
    ('2025-10-31', '2025-10-31'),  # Tots Sants
    ('2025-12-08', '2025-12-09'),  # Immaculada
    ('2025-12-22', '2026-01-07'),  # Vacances de Nadal
    ('2026-02-16', '2026-02-17'),  # Dia de la Comunitat Valenciana
    ('2026-03-30', '2026-04-06'),  # Vacances de Pasqua
    ('2026-04-30', '2026-04-30'),  # Dia del Treball
]

hores_x_dia_de_la_setmana = (
    # dl, dt, dc, dj, dv
    0, 2, 0, 0, 0
)

temes = [
  {
    "tema": "Tema_1",
    "nom": "Llenguatge de Marques",
    "pes": 8,
    "RA": "R1"
  },
  {
    "tema": "Tema_2",
    "nom": "HTML i CSS",
    "pes": 12,
    "RA": "R2"
  },
  {
    "tema": "Tema_3",
    "nom": "JavaScript",
    "pes": 5,
    "RA": "R3"
  },
  {
    "tema": "Tema_4",
    "nom": "Persistència",
    "pes": 7,
    "RA": "R6"
  },
  {
    "tema": "Tema_5",
    "nom": "Validació",
    "pes": 11,
    "RA": "R4"
  },
  {
    "tema": "Tema_6",
    "nom": "Conversió",
    "pes": 11,
    "RA": "R5"
  },
  {
    "tema": "Tema_7",
    "nom": "Sistemes de gestió empresarials",
    "pes": 11,
    "RA": "R7"
  }
]


In [8]:
import pandas as pd
from datetime import datetime, timedelta
import locale

# Configura el locale per a la data en català (pot requerir instal·lació prèvia)
try:
    locale.setlocale(locale.LC_TIME, 'ca_ES.UTF-8')
except locale.Error:
    pass  # Si no està disponible, es mostrarà en format per defecte

# Funció per obtenir llista de dies lectius
def dies_lectius(dia_inici, dia_fi, festius, hores_x_dia):
    dies = []
    data = dia_inici
    while data <= dia_fi:
        dia_setmana = data.weekday()  # 0=dl, 1=dt, ... 4=dv, 5=ds, 6=dg
        if 0 <= dia_setmana <= 4 and hores_x_dia[dia_setmana] > 0:
            festiu = False
            for f_inici, f_fi in festius:
                if f_inici <= data <= f_fi:
                    festiu = True
                    break
            if not festiu:
                dies.append((data, hores_x_dia[dia_setmana]))
        data += timedelta(days=1)
    return dies

# Converteix cadenes a dates
dia_inici = datetime.strptime(dia_inici_classes, '%Y-%m-%d')
dia_fi = datetime.strptime(dia_final_classes, '%Y-%m-%d')
festius_dt = [(datetime.strptime(f[0], '%Y-%m-%d'), datetime.strptime(f[1], '%Y-%m-%d')) for f in festius]

# Obté dies lectius i hores per dia
dies_hores = dies_lectius(dia_inici, dia_fi, festius_dt, hores_x_dia_de_la_setmana)
hores_totals = sum(h for _, h in dies_hores)

# Calcula el total de pes
pes_total = sum(t['pes'] for t in temes)

# Assigna hores proporcionals a cada tema
hores_per_tema = []
for t in temes:
    hores = round(t['pes'] / pes_total * hores_totals)
    hores_per_tema.append(hores)

# Ajusta per assegurar que la suma sigui igual a hores_totals
diferencia = hores_totals - sum(hores_per_tema)
for i in range(abs(diferencia)):
    idx = i % len(hores_per_tema)
    if diferencia > 0:
        hores_per_tema[idx] += 1
    elif diferencia < 0 and hores_per_tema[idx] > 0:
        hores_per_tema[idx] -= 1

# Assigna dies als temes segons les hores calculades
resultats = []
idx = 0
for tema, hores_tema in zip(temes, hores_per_tema):
    hores_assignades = 0
    dies_tema = []
    while hores_assignades < hores_tema and idx < len(dies_hores):
        dia, hores = dies_hores[idx]
        hores_a_assignar = min(hores, hores_tema - hores_assignades)
        hores_assignades += hores_a_assignar
        dies_tema.append(dia)
        idx += 1
    if dies_tema:
        data_inici = dies_tema[0]
        data_fi = dies_tema[-1]
    else:
        data_inici = data_fi = None
    resultats.append({
        'TEMA': tema['tema'],
        'RA': tema['RA'],
        'Nom': tema['nom'],
        'Hores': hores_tema,
        'Data inici': data_inici,
        'Data fi': data_fi
    })

# Mostra la taula
df = pd.DataFrame(resultats)
def format_data(d):
    if d is None: return ''
    try:
        return d.strftime('%-d de %B de %Y')
    except:
        return d.strftime('%d/%m/%Y')

df['Data inici'] = df['Data inici'].apply(format_data)
df['Data fi'] = df['Data fi'].apply(format_data)
df['Data inici - Data fi'] = df['Data inici'] + ' - ' + df['Data fi']
df = df[['TEMA', 'RA', 'Nom', 'Hores', 'Data inici - Data fi']]

In [9]:
print(df.to_markdown(index=False))

| TEMA   | RA   | Nom                             |   Hores | Data inici - Data fi                                  |
|:-------|:-----|:--------------------------------|--------:|:------------------------------------------------------|
| Tema_1 | R1   | Llenguatge de Marques           |       5 | 16 de de setembre de 2025 - 30 de de setembre de 2025 |
| Tema_2 | R2   | HTML i CSS                      |      10 | 7 de d’octubre de 2025 - 4 de de novembre de 2025     |
| Tema_3 | R3   | JavaScript                      |       4 | 11 de de novembre de 2025 - 18 de de novembre de 2025 |
| Tema_4 | R6   | Persistència                    |       6 | 25 de de novembre de 2025 - 16 de de desembre de 2025 |
| Tema_5 | R4   | Validació                       |       9 | 13 de de gener de 2026 - 10 de de febrer de 2026      |
| Tema_6 | R5   | Conversió                       |       9 | 24 de de febrer de 2026 - 24 de de març de 2026       |
| Tema_7 | R7   | Sistemes de gestió empresarials |     